# 挂载google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpa6h3ewjm/pubring.gpg' created
gpg: /tmp/tmpa6h3ewjm/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
!mkdir -p drive 
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!pwd
import os
os.chdir('./drive/COMP5318/comp5318 assignment2/')
!pwd
!ls

/content
/content/drive/COMP5318/comp5318 assignment2
comp5318_assignment2_color_LR.ipynb  dataset
comp5318_assignment2_RF.ipynb	     drive


#assignment

##预处理1

In [0]:
from __future__ import print_function
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix
from scipy.io import loadmat as load

读取文件

In [0]:
train= load('./dataset/train_32x32.mat') # data is train['X'], label is train['y']

分开数据和label

In [0]:
# 分割train_data 成 10个子样本
def seprate_data(data, K, type='data'):
    if type == 'data':
        number_of_training_data = data.shape[3]
        data = data.transpose((3, 0, 1, 2))

    else:
        number_of_training_data = data.shape[0]
        data = data.flatten()
        data[data == 10] = 0

    return data


train_data_t = seprate_data(train['X'], 10, type='data')
print('train_data_t的完整结构', train_data_t.shape)
train_labels_t = seprate_data(train['y'], 10, type='label')
# train_labels_t=np.array(train_labels_t)
print('train_labels_t的完整结构', np.shape(train_labels_t))

train_data_t的完整结构 (73257, 32, 32, 3)
train_labels_t的完整结构 (73257,)


变成灰度

In [0]:
image_size = 32  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def im2gray(image):
    '''Normalize images'''
    image = image.astype(float)
    # Use the Conversion Method in This Paper:
    # [http://www.eyemaginary.com/Rendering/TurnColorsGray.pdf]
    image_gray = np.dot(image, [[0.2989],[0.5870],[0.1140]])
    return image_gray

train_data_c = im2gray(np.array(train_data_t))[:,:,:,0]

print('train_data_c的完整结构',train_data_c.shape)

train_data_c的完整结构 (73257, 32, 32)


正则化

In [0]:
def GCN(image, min_divisor=1e-4):
    """Global Contrast Normalization"""

    imsize = image.shape[0]
    mean = np.mean(image, axis=(1, 2), dtype=float)
    std = np.std(image, axis=(1, 2), dtype=float, ddof=1)
    std[std < min_divisor] = 1.
    image_GCN = np.zeros(image.shape, dtype=float)

    for i in np.arange(imsize):
        image_GCN[i, :, :] = (image[i, :, :] - mean[i]) / std[i]

    return image_GCN


train_data_GCN = GCN(train_data_c)

print(train_data_GCN.shape, np.shape(train_labels_t))

(73257, 32, 32) (73257,)


##创建训练集和测试集

In [0]:
def train_test_split_10(train_X, train_y, k):
    num_of_examples = len(train_X)
    start_jump = (num_of_examples // 10) * k
    end_jump = (num_of_examples // 10) * (k + 1)
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for i in range(len(train_X)):
        if i <= start_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        elif i > end_jump:
            X_train.append(train_X[i])
            y_train.append(train_y[i])
        else:
            X_test.append(train_X[i])
            y_test.append(train_y[i])

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    return X_train,  y_train,X_test, y_test

train_data_GCN,train_labels,test_data_GCN,test_labels=train_test_split_10(train_data_GCN, train_labels_t,0)
print(np.shape(train_data_GCN))
print(np.shape(train_labels))
print(np.shape(test_data_GCN))
print(np.shape(test_labels))


(65932, 32, 32)
(65932,)
(7325, 32, 32)
(7325,)


##SGD分类器

In [0]:
clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, shuffle=True, verbose=0,\
                    n_jobs=4, random_state=None, learning_rate='optimal')
clf.fit(train_data_GCN.reshape(train_data_GCN.shape[0],-1), train_labels)
train_prediction = clf.predict(train_data_GCN.reshape(train_data_GCN.shape[0],-1))
valid_prediction = clf.predict(test_data_GCN.reshape(test_data_GCN.shape[0],-1))

print('Training score is', clf.score(train_data_GCN.reshape(train_data_GCN.shape[0],-1), train_labels))
print('Validation score is', clf.score(test_data_GCN.reshape(test_data_GCN.shape[0],-1), test_labels))

print('Classification report of training data:\n', classification_report(train_labels, train_prediction))
print('Confusion Matrix of training data:\n', confusion_matrix(train_labels, train_prediction))

print('Classification report of validation data:\n', classification_report(test_labels, valid_prediction))
print('Confusion Matrix of validation data:\n', confusion_matrix(test_labels, valid_prediction))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Training score is 0.20751683552751318
Validation score is 0.19467576791808874
Classification report of training data:
              precision    recall  f1-score   support

          0       0.55      0.08      0.14      4434
          1       0.21      0.44      0.29     12457
          2       0.17      0.40      0.24      9554
          3       0.16      0.26      0.20      7667
          4       0.39      0.10      0.16      6671
          5       0.52      0.07      0.13      6181
          6       0.46      0.05      0.09      5166
          7       0.40      0.04      0.08      5068
          8       0.22      0.03      0.06      4530
          9       0.42      0.05      0.09      4204

avg / total       0.32      0.21      0.17     65932

Confusion Matrix of training data:
 [[ 369 1807 1460  619   78    8   21   35   34    3]
 [  28 5542 3644 2797  190   57   49   65   53   32]
 [  77 3372 3818 1852  149   38   36   43   99   70]
 [  40 2866 2330 1992  126  105    9   33  128 